# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [14]:
%help


# Available Magic Commands

## Sessions Magic

----
    %help                             Return a list of descriptions and input types for all magic commands. 
    %profile            String        Specify a profile in your aws configuration to use as the credentials provider.
    %region             String        Specify the AWS region in which to initialize a session. 
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\ USERNAME \.aws\config" on Windows.
    %idle_timeout       Int           The number of minutes of inactivity after which a session will timeout. 
                                      Default: 2880 minutes (48 hours).
    %session_id_prefix  String        Define a String that will precede all session IDs in the format 
                                      [session_id_prefix]-[session_id]. If a session ID is not provided,
                                      a random UUID will be generated.
    %status                           Returns the status of the current Glue session including its duration, 
                                      configuration and executing user / role.
    %session_id                       Returns the session ID for the running session. 
    %list_sessions                    Lists all currently running sessions by ID.
    %stop_session                     Stops the current session.
    %glue_version       String        The version of Glue to be used by this session. 
                                      Currently, the only valid options are 2.0, 3.0 and 4.0. 
                                      Default: 2.0.
----

## Selecting Job Types

----
    %streaming          String        Sets the session type to Glue Streaming.
    %etl                String        Sets the session type to Glue ETL.
    %glue_ray           String        Sets the session type to Glue Ray.
----

## Glue Config Magic 
*(common across all job types)*

----

    %%configure         Dictionary    A json-formatted dictionary consisting of all configuration parameters for 
                                      a session. Each parameter can be specified here or through individual magics.
    %iam_role           String        Specify an IAM role ARN to execute your session with.
                                      Default from ~/.aws/config on Linux or macOS, 
                                      or C:\Users\%USERNAME%\.aws\config` on Windows.
    %number_of_workers  int           The number of workers of a defined worker_type that are allocated 
                                      when a session runs.
                                      Default: 5.
    %additional_python_modules  List  Comma separated list of additional Python modules to include in your cluster 
                                      (can be from Pypi or S3).
    %%tags        Dictionary          Specify a json-formatted dictionary consisting of tags to use in the session.
----

                                      
## Magic for Spark Jobs (ETL & Streaming)

----
    %worker_type        String        Set the type of instances the session will use as workers. 
                                      ETL and Streaming support G.1X, G.2X, G.4X and G.8X. 
                                      Default: G.1X.
    %connections        List          Specify a comma separated list of connections to use in the session.
    %extra_py_files     List          Comma separated list of additional Python files From S3.
    %extra_jars         List          Comma separated list of additional Jars to include in the cluster.
    %spark_conf         String        Specify custom spark configurations for your session. 
                                      E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer
----
                                      
## Magic for Ray Job

----
    %min_workers        Int           The minimum number of workers that are allocated to a Ray job. 
                                      Default: 1.
    %object_memory_head Int           The percentage of free memory on the instance head node after a warm start. 
                                      Minimum: 0. Maximum: 100.
    %object_memory_worker Int         The percentage of free memory on the instance worker nodes after a warm start. 
                                      Minimum: 0. Maximum: 100.
----

## Action Magic

----

    %%sql               String        Run SQL code. All lines after the initial %%sql magic will be passed
                                      as part of the SQL code.  
----



####  Run this cell to set up and start your interactive session.


In [2]:
!pip install openpyxl

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 6.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [20]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 3
%additional_python_modules openpyxl,loguru

You are already connected to a glueetl session 73f7459c-05a3-4172-bcab-7153aafb9c88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 73f7459c-05a3-4172-bcab-7153aafb9c88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 73f7459c-05a3-4172-bcab-7153aafb9c88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 73f7459c-05a3-4172-bcab-7153aafb9c88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 3
Setting new number of workers to: 3


You are already connected to a glueetl session 73f7459c-05a3-4172-bcab-7153aafb9c88.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Additional python modules to be included:
openpyxl
loguru


In [ ]:
import sys
from pathlib import Path

import pandas as pd
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import DataFrame as SparkDataFrame

In [12]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

## FUNÇÕES GLOBAIS

In [3]:
def convert_dataframe(dataframe, target_framework, spark_session=None):
    """
    Converte entre pandas DataFrame, PySpark DataFrame e AWS Glue DynamicFrame.

    :param dataframe: DataFrame a ser convertido.
    :param target_framework: O tipo de estrutura de dados de destino ("pandas", "spark" ou "dynamicframe").
    :param spark_session: Sessão Spark (usado apenas se target_framework for "spark").
    :return: DataFrame ou DynamicFrame, conforme o tipo de estrutura de destino especificado.
    """

    if target_framework == "pandas":
        if isinstance(dataframe, SparkDataFrame):
            return dataframe.toPandas()
        elif isinstance(dataframe, DynamicFrame):
            return dataframe.toDF().toPandas()  # Convertendo DynamicFrame para Spark DataFrame e depois para pandas
        else:
            return dataframe

    elif target_framework == "spark":
        if isinstance(dataframe, pd.DataFrame):
            return spark_session.createDataFrame(dataframe)
        elif isinstance(dataframe, DynamicFrame):
            return dataframe.toDF()  # Convertendo DynamicFrame diretamente para Spark DataFrame
        else:
            return dataframe

    elif target_framework == "dynamicframe":
        if isinstance(dataframe, pd.DataFrame):
            spark_dataframe = spark_session.createDataFrame(dataframe)
            return DynamicFrame.fromDF(spark_dataframe, glueContext, "dynamicframe")
        elif isinstance(dataframe, SparkDataFrame):
            return DynamicFrame.fromDF(dataframe, glueContext, "dynamicframe")
        else:
            return dataframe

    else:
        raise ValueError("Invalid target_framework.")

In [4]:
def read_generic_data(file_path, format_type=None, sheet_name=None, separator=None, spark=None):
    """
    Lê dados de vários formatos usando PySpark ou pandas.

    :param file_path: Caminho do arquivo.
    :param format_type: Tipo de formato ("csv", "parquet", "json", "excel").
    :param sheet_name: Nome da planilha (apenas para format_type="excel").
    :param separator: Separador de colunas (apenas para format_type="csv").
    :param spark: Instância do SparkSession (opcional).
    :return: DataFrame (Spark DataFrame ou pandas DataFrame, dependendo do caso).
    """

    if format_type is None:
        format_type = Path(file_path_csv).suffix[1:].lower()

    if format_type in ["csv", "parquet", "json"]:
        if spark:
            if format_type == "csv":
                if separator is None:
                    separator = ","
                return spark.read.csv(file_path, header=True, inferSchema=True, sep=separator)
            elif format_type == "parquet":
                return spark.read.parquet(file_path)
            elif format_type == "json":
                return spark.read.json(file_path)
        else:
            if format_type == "csv":
                if separator is None:
                    separator = ","
                return pd.read_csv(file_path, sep=separator)
            elif format_type == "parquet":
                return pd.read_parquet(file_path)
            elif format_type == "json":
                return pd.read_json(file_path)
    elif format_type in ["excel", "xlsx"]:
        if sheet_name is None:
            sheet_name = 0
            
        # READING EXCEL IN PANDAS DATAFRAME
        df_pandas = pd.read_excel(file_path, 
                                  sheet_name=sheet_name, 
                                  engine="openpyxl")
        
        if spark:
            convert_dataframe(dataframe=df_pandas, 
                              target_framework="spark", 
                              spark_session=spark)
        
    else:
        raise ValueError("Format not supported.")

## DADOS EM FORMATO CSV

In [12]:
file_path_csv = "s3://etl-customer-orders/data/raw_data/orders.csv"

In [16]:
df_csv = read_generic_data(file_path=file_path_csv, 
                           format_type="csv", 
                           separator=";", 
                           spark=spark)

In [17]:
df_csv.show()

+-------+----------+----------+-------------------+-------------------+-------------------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|OrderID|CustomerID|EmployeeID|          OrderDate|       RequiredDate|        ShippedDate|ShipVia|Freight|            ShipName|         ShipAddress|      ShipCity|ShipRegion|ShipPostalCode|ShipCountry|
+-------+----------+----------+-------------------+-------------------+-------------------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|  10248|     VINET|         5|2020-07-04 00:00:00|2020-08-01 00:00:00|2020-07-16 00:00:00|      3|  32.38|Vins et alcools C...|  59 rue de l-Abbaye|         Reims|      null|         51100|     France|
|  10249|     TOMSP|         6|2020-07-05 00:00:00|2020-08-16 00:00:00|2020-07-10 00:00:00|      1|  11.61|   Toms Spezialitten|       Luisenstr. 48|        Mnster|      null|         4408

## DADOS EM FORMATO EXCEL

In [18]:
file_path_excel = "s3://etl-customer-orders/data/raw_data/BASE_COM_CEP.xlsx"

In [21]:
df_excel = read_generic_data(file_path=file_path_excel, 
                             spark=spark)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.


In [22]:
df_excel.show()

+-------+----------+----------+-------------------+-------------------+-------------------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|OrderID|CustomerID|EmployeeID|          OrderDate|       RequiredDate|        ShippedDate|ShipVia|Freight|            ShipName|         ShipAddress|      ShipCity|ShipRegion|ShipPostalCode|ShipCountry|
+-------+----------+----------+-------------------+-------------------+-------------------+-------+-------+--------------------+--------------------+--------------+----------+--------------+-----------+
|  10248|     VINET|         5|2020-07-04 00:00:00|2020-08-01 00:00:00|2020-07-16 00:00:00|      3|  32.38|Vins et alcools C...|  59 rue de l-Abbaye|         Reims|      null|         51100|     France|
|  10249|     TOMSP|         6|2020-07-05 00:00:00|2020-08-16 00:00:00|2020-07-10 00:00:00|      1|  11.61|   Toms Spezialitten|       Luisenstr. 48|        Mnster|      null|         4408

In [ ]:
type(df_excel)